In [7]:
from ultralytics import YOLO
import torch
from operator import itemgetter

# 1. โหลดโมเดล
model = YOLO('./models/yolo8n_best.pt')

# 2. เตรียม label_dict
label_dict = {}

# ตัวเลข 0-9
for i in range(10):
    label_dict[str(i)] = str(i)

# ตัวอักษร ก-ฮ (A01-A44)
for i in range(1, 45):
    key = f"A{str(i).zfill(2)}"
    value = chr(ord('ก') + i - 1)
    label_dict[key] = value

# จังหวัด
province_map = {
    "KBI": "กระบี่", "KRI": "กาญจนบุรี", "KSN": "กาฬสินธุ์", "KPT": "กำแพงเพชร", "KKN": "ขอนแก่น",
    "CTI": "จันทบุรี", "CCO": "ฉะเชิงเทรา", "CBI": "ชลบุรี", "CNT": "ชัยนาท", "CPM": "ชัยภูมิ",
    "CPN": "ชุมพร", "CRI": "เชียงราย", "CMI": "เชียงใหม่", "TRG": "ตรัง", "TRT": "ตราด",
    "TAK": "ตาก", "NYK": "นครนายก", "NPT": "นครปฐม", "NPM": "นครพนม", "NMA": "นครราชสีมา",
    "NRT": "นครศรีธรรมราช", "NSN": "นครสวรรค์", "NBI": "นนทบุรี", "NWT": "นราธิวาส", "NAN": "น่าน",
    "BKN": "บึงกาฬ", "BRM": "บุรีรัมย์", "PTE": "ปทุมธานี", "PKN": "ประจวบคีรีขันธ์", "PRI": "ปราจีนบุรี",
    "PTN": "ปัตตานี", "PYO": "พะเยา", "AYA": "พระนครศรีอยุธยา", "PNA": "พังงา", "PLG": "พัทลุง",
    "PCT": "พิจิตร", "PLK": "พิษณุโลก", "PBI": "เพชรบุรี", "PNB": "เพชรบูรณ์", "PRE": "แพร่",
    "PKT": "ภูเก็ต", "MKM": "มหาสารคาม", "MDH": "มุกดาหาร", "MSN": "แม่ฮ่องสอน", "YST": "ยโสธร",
    "YLA": "ยะลา", "RET": "ร้อยเอ็ด", "RNG": "ระนอง", "RYG": "ระยอง", "RBR": "ราชบุรี",
    "LRI": "ลพบุรี", "LPG": "ลำปาง", "LPN": "ลำพูน", "LEI": "เลย", "SSK": "ศรีสะเกษ",
    "SNK": "สกลนคร", "SKA": "สงขลา", "STN": "สตูล", "SPK": "สมุทรปราการ", "SKM": "สมุทรสงคราม",
    "SKN": "สมุทรสาคร", "SKW": "สระแก้ว", "SRI": "สระบุรี", "SBR": "สิงห์บุรี", "STI": "สุโขทัย",
    "SPB": "สุพรรณบุรี", "SNI": "สุราษฎร์ธานี", "SRN": "สุรินทร์", "NKI": "หนองคาย", "NBP": "หนองบัวลำภู",
    "ATG": "อ่างทอง", "ACR": "อำนาจเจริญ", "UDN": "อุดรธานี", "UTT": "อุตรดิตถ์", "UTI": "อุทัยธานี",
    "UBN": "อุบลราชธานี", "BKK": "กรุงเทพมหานคร"
}
label_dict.update(province_map)

# 3. ฟังก์ชันถอดรหัสป้ายทะเบียน
def decode_license_plate(result, class_names, label_dict, province_map):
    boxes = result.boxes
    if boxes is None or boxes.cls is None:
        return "[ไม่พบกล่องตรวจจับ]"

    cls_list = boxes.cls.cpu().numpy()
    x_centers = boxes.xywh[:, 0].cpu().numpy()

    detections = [(class_names[int(cls)], float(x)) for cls, x in zip(cls_list, x_centers)]
    sorted_detections = sorted(detections, key=itemgetter(1))

    decoded = ""
    province = ""

    for cls_name, _ in sorted_detections:
        if cls_name in province_map:
            province = label_dict.get(cls_name, f"[{cls_name}]")
        else:
            decoded += label_dict.get(cls_name, f"[{cls_name}]")

    return decoded + province

# 4. รันภาพเดียว (หรือจะใส่ list รูปก็ได้)
image_path = '1731240159727_lp0.jpg'
results = model(image_path)

# 5. แสดงผล
for result in results:
    result.show()  # แสดงภาพพร้อมกล่อง
    license_plate = decode_license_plate(result, model.names, label_dict, province_map)
    print("ทะเบียนรถที่ถอดรหัสได้:", license_plate)



image 1/1 /Users/watchara/Documents/me/License-Plate-Recognition-Lab/1731240159727_lp0.jpg: 384x640 1 2, 1 3, 1 4, 2 7s, 1 A18, 1 A41, 1 BKK, 48.5ms
Speed: 2.0ms preprocess, 48.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
ทะเบียนรถที่ถอดรหัสได้: 3ฒษ2477กรุงเทพมหานคร


In [14]:
from ultralytics import YOLO
import cv2
import torch
import easyocr
from operator import itemgetter

# 1. โหลดโมเดล
yolo_det = YOLO('./models/license_plate_detector.pt')      # โมเดล detect ป้าย
yolo_ocr = YOLO('./models/yolo8x_best.pt')         # โมเดลอ่านตัวอักษร
reader  = easyocr.Reader(['th','en'], gpu=torch.cuda.is_available())

# 2. ฟังก์ชัน OCR แต่ละตัว
def ocr_easyocr(img_crop):
    rgb = cv2.cvtColor(img_crop, cv2.COLOR_BGR2RGB)
    texts = reader.readtext(rgb, detail=0)
    return ''.join(texts)

def ocr_yolo(result, class_names, label_dict, province_map):
    # ใช้ decode_license_plate เดิมของคุณ
    return decode_license_plate(result, class_names, label_dict, province_map)

# 3. อ่านภาพ, detect ป้าย แล้ว crop
img = cv2.imread('1744878567335_lp0.jpg')
det_results = yolo_det(img)

for det in det_results:
    for box, cls in zip(det.boxes.xyxy, det.boxes.cls):
        # สมมติ cls==0 คือป้ายทะเบียน
        if int(cls)==0:
            x1,y1,x2,y2 = map(int, box.cpu().numpy())
            plate = img[y1:y2, x1:x2]

            # 4. รัน OCR ทั้ง 3 แหล่ง
            # 4.1 YOLO OCR โมเดล
            ocr_results = yolo_ocr(plate)  
            yolo_text = ''
            for r in ocr_results:
                yolo_text = ocr_yolo(r, yolo_ocr.names, label_dict, province_map)

            # 4.2 EasyOCR
            easy_text = ocr_easyocr(plate)

            # 4.3 (Optional) รวม output YOLO OCR ที่คุณมีเดิม
            # decoded = decode_license_plate(det, yolo_det.names, label_dict, province_map)

            # 5. แสดงผล cross-check
            print(">> YOLO-OCR model:", yolo_text)
            print(">> EasyOCR      :", easy_text)
            # print(">> YOLO-decode  :", decoded)
            print("-----------------------------")


Using CPU. Note: This module is much faster with a GPU.



0: 352x640 1 license_plate, 20.5ms
Speed: 2.1ms preprocess, 20.5ms inference, 0.4ms postprocess per image at shape (1, 3, 352, 640)

0: 288x640 1 0, 1 2, 2 3s, 1 4, 1 A18, 1 A44, 1 BKK, 174.7ms
Speed: 0.6ms preprocess, 174.7ms inference, 0.4ms postprocess per image at shape (1, 3, 288, 640)
>> YOLO-OCR model: 3ฒฬ2430กรุงเทพมหานคร
>> EasyOCR      : 3๗|ฮ 2430กรุงเทพมหานคร tovota krungthai
-----------------------------
